In [2]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [ ]:
# Libraries
import os
import sys
IntialiseEnv()
import tensorflow as tf
import numpy as np
import math
import pickle
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [8]:
# Define paths
data_dir = r"D:\GP_datasets\archive(1)"
train_path = os.path.join(data_dir, "Train")
test_path = os.path.join(data_dir, "Test")

# Check if the paths exist
print(f"Train directory exists: {os.path.exists(train_path)}")
print(f"Test directory exists: {os.path.exists(test_path)}")

# Define image dimensions for LeNet5 (traditionally 32x32)
IMG_HEIGHT = 32
IMG_WIDTH = 32
CHANNELS = 3

Train directory exists: True
Test directory exists: True


In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers

def build_lenet5_model(input_shape=(32, 32, 3), num_classes=43):
    """
    Build LeNet5 model for traffic sign classification
    
    Parameters:
    - input_shape: shape of input images (height, width, channels)
    - num_classes: number of traffic sign classes
    
    Returns:
    - model: compiled LeNet5 model
    """
    model = models.Sequential([
        # Layer 1: Convolutional + ReLU + Pooling
        layers.Conv2D(6, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        # Layer 2: Convolutional + ReLU + Pooling
        layers.Conv2D(16, kernel_size=(5, 5), padding='valid', activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        # Flatten
        layers.Flatten(),
        
        # Layer 3: Fully Connected + ReLU
        layers.Dense(120, activation='relu'),
        
        # Layer 4: Fully Connected + ReLU
        layers.Dense(84, activation='relu'),
        
        # Output Layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

In [6]:
def preprocess_data(X_train, X_test):
    """
    Preprocess data for LeNet5 model
    
    Parameters:
    - X_train: training images
    - X_test: testing images
    
    Returns:
    - X_train_normalized: normalized training images
    - X_test_normalized: normalized testing images
    """
    # Resize images to 32x32 if they aren't already
    # (LeNet5 was designed for 32x32 images)
    if X_train.shape[1:3] != (32, 32):
        # Implementation of resizing would go here
        pass
        
    # Normalize pixel values to [0,1]
    X_train_normalized = X_train.astype('float32') / 255.0
    X_test_normalized = X_test.astype('float32') / 255.0
    
    return X_train_normalized, X_test_normalized

In [7]:
def train_model(model, X_train, y_train, X_test, y_test, batch_size=32, epochs=20):
    """
    Train the LeNet5 model
    
    Parameters:
    - model: compiled LeNet5 model
    - X_train, y_train: training data
    - X_test, y_test: test data
    - batch_size: batch size for training
    - epochs: number of training epochs
    
    Returns:
    - history: training history
    - model: trained model
    """
    # Data augmentation to help with generalization
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        shear_range=0.1
    )
    
    # Convert labels to categorical if they aren't already
    if len(y_train.shape) == 1:
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)
    
    # Early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    
    # Train the model
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping]
    )
    
    # Evaluate on test data
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_acc:.4f}")
    
    return history, model

In [11]:
def load_data_from_folders(train_path, test_path):
    """
    Load data from folder structure if pickle files don't exist
    """
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []
    
    # Load training data
    print("Loading training data from folders...")
    for class_id in range(43):  # 43 classes in GTSRB dataset
        class_path = os.path.join(train_path, str(class_id))
        if not os.path.exists(class_path):
            continue
            
        for image_file in os.listdir(class_path):
            if not image_file.endswith('.png') and not image_file.endswith('.jpg') and not image_file.endswith('.ppm'):
                continue
                
            img_path = os.path.join(class_path, image_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
                
            img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            train_data.append(img)
            train_labels.append(class_id)
    
    # Load test data
    print("Loading test data from folders...")
    for class_id in range(43):
        class_path = os.path.join(test_path, str(class_id))
        if not os.path.exists(class_path):
            continue
            
        for image_file in os.listdir(class_path):
            if not image_file.endswith('.png') and not image_file.endswith('.jpg') and not image_file.endswith('.ppm'):
                continue
                
            img_path = os.path.join(class_path, image_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
                
            img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            test_data.append(img)
            test_labels.append(class_id)
    
    # Convert to numpy arrays
    train_data = np.array(train_data)
    train_labels = np.array(train_labels)
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
    print(f"Training data loaded: {train_data.shape}, {train_labels.shape}")
    print(f"Test data loaded: {test_data.shape}, {test_labels.shape}")
    
    return train_data, train_labels, test_data, test_labels

In [ ]:
def load_data_from_pickle():
    """
    Load data from pickle files if they exist
    """
    train_file = os.path.join(data_dir, "Processed_DataSet", "GermanTrainDataSet.pkl")
    test_file = os.path.join(data_dir, "Processed_DataSet", "GermanTestDataSet.pkl")
    
    train_data, train_labels = None, None
    test_data, test_labels = None, None
    
    if os.path.exists(train_file):
        print("Loading training data from pickle file...")
        with open(train_file, 'rb') as f:
            train_data, train_labels = pickle.load(f)
        print(f"Training data loaded: {train_data.shape}, {train_labels.shape}")
    
    if os.path.exists(test_file):
        print("Loading test data from pickle file...")
        with open(test_file, 'rb') as f:
            test_data, test_labels = pickle.load(f)
        print(f"Test data loaded: {test_data.shape}, {test_labels.shape}")
    
    return train_data, train_labels, test_data, test_labels

In [10]:
# Load data
train_data, train_labels, test_data, test_labels = load_data_from_pickle()

# If pickle files don't exist, load from folders
if train_data is None or test_data is None:
    train_data, train_labels, test_data, test_labels = load_data_from_folders(train_path, test_path)

# Check data shapes and types
print(f"Training data shape: {train_data.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Test labels shape: {test_labels.shape}")

# Check unique classes
num_classes = len(np.unique(train_labels))
print(f"Number of unique classes: {num_classes}")


NameError: name 'load_data_from_pickle' is not defined

In [ ]:
# Assuming you have your data loaded as X_train, y_train, X_test, y_test

x

# 1. Preprocess the data
X_train_preprocessed, X_test_preprocessed = preprocess_data(X_train, X_test)

# 2. Build the LeNet5 model
model = build_lenet5_model(input_shape=(32, 32, 3), num_classes=len(np.unique(y_train)))

# 3. Train the model
history, trained_model = train_model(
    model,
    X_train_preprocessed, y_train,
    X_test_preprocessed, y_test,
    batch_size=64,
    epochs=30
)

# 4. Save the trained model
trained_model.save("traffic_sign_lenet5_model.h5")